<a href="https://colab.research.google.com/github/ShinWalnut/ABR_basic/blob/master/RNN_test_hihello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

idx2char = ['h', 'i', 'e', 'l', 'o']   # h->0 , i->1, e->2 ... 에 대응시켜주기위한 인덱스

x_data = [[0, 1, 0, 2, 3, 3]] # 입력으로 hihell을 준다.

x_one_hot = [[[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0]]]

y_data = [[1, 0, 2, 3, 3, 4]] # 출력으로 ihello를 얻고싶다.

num_classes = 5
input_dim = 5
hidden_size = 5
sequence_length = 6
batch_size = 1
learning_rate = 0.1

# X.shape = (batchsize , sequence_length, voca_length)
X = tf.placeholder(dtype=tf.float32, shape=[None, sequence_length, input_dim])
Y = tf.placeholder(dtype=tf.int32, shape=[None, sequence_length])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, state_ = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

#output.shape = (batchsize, sequence_length, hidden_size)
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.layers.dense(inputs=X_for_fc, units=num_classes)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(30):
        l, _ = sess.run([loss,train], feed_dict={X:x_one_hot, Y:y_data})
        result = sess.run(prediction, feed_dict={X:x_one_hot})

        print(i, "loss : ", l, " prediction : ", result, " true Y : ", y_data)
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str : ", "".join(result_str))



Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

0 loss :  1.7616792  prediction :  [[2 2 2 0 1 1]]  true Y :  [[1, 0, 2, 3, 3, 4]]
	Prediction str :  eeehii
1 loss :  1.4215727  prediction :  [[1 0 2 0 1 3]]  true Y :  [[1, 0, 2, 3, 3, 4]]
	Prediction str :  ihehil
2 loss :  1.2065637  prediction :  [[1 0 2 3 3 3]]  true Y :  [[1, 0, 2, 3, 3, 4]]
	Prediction str :  ihelll
3 loss :  1.054863  prediction :  [[1 0 2 3 3 3]]  true Y :  [[1, 0, 2, 3, 3